In [299]:
import pandas as pd
import numpy as np
import re

#Webscraping
import requests
from requests import get
from bs4 import BeautifulSoup
import time

In [300]:
df = pd.read_csv('data/michelin_my_maps.csv')

In [301]:
df.head(2)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count..."


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6780 entries, 0 to 6779
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   6780 non-null   object 
 1   Address                6780 non-null   object 
 2   Location               6780 non-null   object 
 3   Price                  6726 non-null   object 
 4   Cuisine                6780 non-null   object 
 5   Longitude              6780 non-null   float64
 6   Latitude               6780 non-null   float64
 7   PhoneNumber            6649 non-null   float64
 8   Url                    6780 non-null   object 
 9   WebsiteUrl             5542 non-null   object 
 10  Award                  6780 non-null   object 
 11  FacilitiesAndServices  6732 non-null   object 
dtypes: float64(3), object(9)
memory usage: 635.8+ KB


In [263]:
df.Award.value_counts()

Bib Gourmand        3411
1 MICHELIN Star     2744
2 MICHELIN Stars     483
3 MICHELIN Stars     142
Name: Award, dtype: int64

In [264]:
df.Url[2]

'https://guide.michelin.com/en/shanghai-municipality/shanghai/restaurant/taian-table'

In [72]:
#Make all column names lowercase
df.columns = df.columns.str.lower()

In [73]:
#renaming columns in snake case
df = df.rename(columns = {"phonenumber":"phone_number", "websiteurl":"website_url", "facilitiesandservices":"facilities_and_services"})

In [74]:
#Calling restauraunt by name -- Using conventional capitalization
df[df.name == 'Rooster and the Till']

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services
5409,Rooster and the Till,"6500 N. Florida Ave., Tampa, 33604, USA","Tampa, USA",$$,International,-82.459512,28.006834,1.813375e+10,https://guide.michelin.com/en/florida/tampa/re...,https://www.roosterandthetill.com/,Bib Gourmand,"Air conditioning,American Express credit card,..."


In [75]:
df.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count..."


In [76]:
df.url[0]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [77]:
df.url[1]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/xin-rong-ji-xinyuan-south-road'

In [15]:
# parts = url_0.split("en/")
# parts[1]

'beijing-municipality/beijing/restaurant/king-s-joy'

In [16]:
# df = df.assign(parts= df.url.str.split("en/").str[1])

In [80]:
df.url

0       https://guide.michelin.com/en/beijing-municipa...
1       https://guide.michelin.com/en/beijing-municipa...
2       https://guide.michelin.com/en/shanghai-municip...
3       https://guide.michelin.com/en/shanghai-municip...
4       https://guide.michelin.com/en/california/san-f...
                              ...                        
6775    https://guide.michelin.com/en/masovia/warsaw/r...
6776    https://guide.michelin.com/en/lesser-poland/cr...
6777    https://guide.michelin.com/en/lesser-poland/cr...
6778    https://guide.michelin.com/en/masovia/warsaw/r...
6779    https://guide.michelin.com/en/masovia/warsaw/r...
Name: url, Length: 6780, dtype: object

In [209]:
df.url[0]

'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [302]:
### Using a test df to troubleshoot
df = pd.read_pickle('data/dftest5.pickle')

In [303]:
df.head(2)

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...


In [304]:
#Hardcoding the url for first restaurant
url = 'https://guide.michelin.com/en/beijing-municipality/beijing/restaurant/king-s-joy'

In [305]:
requests.get(url)
response = requests.get(url)
response

<Response [200]>

In [269]:
#telling it you will feed it html
soup = BeautifulSoup(response.content, 'html.parser')

In [270]:
king = soup.find('div', class_='restaurant-details__description--text')
king.find('p', {'class':None} ).text.strip()
#king.find_all('p', class_="{'class':None}")

'Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.'

In [171]:
soup.find_all('p', class_='restaurant-details__likeinfo--description pl-text js-favorite__description')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>]

In [172]:
soup.find_all('p')

[<p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p class="restaurant-details__likeinfo--description pl-text js-favorite__description">
             Add to favorites
         </p>,
 <p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>,
 <p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to

In [173]:
soup.select('p:not([class])')

[<p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>,
 <p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>,
 <p>"We offer vegetarian cuisine using green and seasonal produce that is sourced from lo

In [174]:
soup.find('p', {'class':None})

<p><p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p></p>

In [175]:
soup.find('p', {'class':None}).text.strip()

'Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.'

In [176]:
soup.find('p', {'class':None}).contents[0]

<p>Just a stone’s throw from Yonghe Temple, this serene restaurant is remodelled from a courtyard house and features a glass-roofed dining room. The chef honed his skills at his grandparents’ restaurant of the same name. Only set menus are available; try the classic menu to sample their specialities all at once. Regulars may opt for the solar-term menu that changes every 15 days. Live harp performance befits the restaurant’s classy elegance.</p>

In [177]:
df['url']

0    https://guide.michelin.com/en/beijing-municipa...
1    https://guide.michelin.com/en/beijing-municipa...
2    https://guide.michelin.com/en/shanghai-municip...
3    https://guide.michelin.com/en/shanghai-municip...
4    https://guide.michelin.com/en/california/san-f...
Name: url, dtype: object

In [294]:
testdf = pd.read_pickle('data/dftest5.pickle')
testdf

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince


In [298]:
df.url[0]

''

In [292]:
#Test for first url
output= []
soup = BeautifulSoup(requests.get(page).content)
more_soup = soup.find('div', class_='restaurant-details__description--text')
page_output = more_soup.find('p', {'class':None} ).text.strip()
output.append(page_output)
# time.sleep(2)
#df['data'].loc[0] = output

In [293]:
output

["An air of refinement infuses this dining room, thanks to a Murano chandelier, stylish diners and everything in between. No wonder it's where affluent tourists and locals come to celebrate special occasions. From the moment that champagne cart arrives to the last bite of the guéridon’s mignardises, service is wonderfully paced.The menu unfolds as a series of compositions that are laser-focused on product; say a petite bowl of perfect chanterelle mushrooms arranged with shaved ribbons of celtuce and slicked with lardo. Pasta has always been the chef's forte, and newer creations have included a daring twirl of beet-red artisanal spaghetti with Tsar Nicoulai caviar and sturgeon crema or a divine bowlful of agnolottini verdi filled with smoked squab."]

In [291]:
df['data']

0    [An air of refinement infuses this dining room...
1    This branch of the chain restaurant opened in ...
2    A fixture on the city’s dining scene since 201...
3    This multi-sensory experience at a secret loca...
4    An air of refinement infuses this dining room,...
Name: data, dtype: object

In [97]:
# def get_michelin_pages():
#     df = pd.read_pickle('data/dftest5.pickle')
#     urls = df['url']
#     output = []
#     michelin_df = pd.DataFrame(columns=['data'])
#     for page in urls:
#         try:
#             soup = BeautifulSoup(requests.get(page).content, 'html.parser')
#             page_output = soup.find('p', {'class':None}).text.strip()
#             output.append(page_output)
#             time.sleep(2)
#             michelin_df['data'].loc[page] = output
#         except:
#             pass
#     michelin_df.to_pickle('data/michelin_df.pickle')

In [98]:
get_michelin_pages()

In [101]:
# def get_michelin_pages():
#     df = pd.read_pickle('data/dftest5.pickle')
#     urls = df['url']
#     output = []
#     michelin_df = pd.DataFrame(columns=['data'])
#     for page in urls:
#         response = requests.get(page)
#         soup = BeautifulSoup(response.content, 'html.parser')
#         more_soup = soup.find('div', class_='restaurant-details__description--text')
#         page_output = more_soup.find('p', {'class':None} ).text.strip()
#         output.append(page_output)
#         time.sleep(2)
#         michelin_df['data'].loc[page] = output
#     michelin_df.to_pickle('michelin_df.pickle')

In [105]:
df = pd.read_pickle('data/dftest5.pickle')

In [107]:
urls = df['url']

In [151]:
#Its a series
urls

0    https://guide.michelin.com/en/beijing-municipa...
1    https://guide.michelin.com/en/beijing-municipa...
2    https://guide.michelin.com/en/shanghai-municip...
3    https://guide.michelin.com/en/shanghai-municip...
4    https://guide.michelin.com/en/california/san-f...
Name: url, dtype: object

In [111]:
output = []

In [112]:
michelin_df = pd.DataFrame(columns=['data'])

In [113]:
michelin_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    0 non-null      object
dtypes: object(1)
memory usage: 0.0+ bytes


In [289]:
df.url[0] = ''

/var/folders/vw/1h05rsrx3wbc05jyk4c2bl680000gn/T/ipykernel_50191/2049498423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.url[0] = ''


In [256]:
#Calling test df to test this for problems
df = pd.read_pickle('data/dftest5.pickle')

In [307]:
#Trying this one, just getting it to append back to the dataframe
def get_michelin_pages():
    df = pd.read_pickle('data/dftest5.pickle')
    urls = df['url']
    output = []
    #michelin_df = pd.DataFrame(columns=['data'])
    for page in urls:
        try:
            request = requests.get(page)
            soup = BeautifulSoup(request.content)
            more_soup = soup.find('div', class_='restaurant-details__description--text')
            page_output = more_soup.find('p', {'class':None} ).text.strip()
            output.append(page_output)
            time.sleep(2)
            #michelin_df['data'].loc[page] = output
        except requests.exceptions.RequestException as e:
            print(e)
            continue
    #michelin_df.to_pickle('data/michelin_df.pickle')
    df['data'] = output
    return df

In [308]:
get_michelin_pages()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,parts,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",beijing-municipality/beijing/restaurant/king-s...,
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",beijing-municipality/beijing/restaurant/xin-ro...,This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/taia...,A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",shanghai-municipality/shanghai/restaurant/ultr...,This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",california/san-francisco/restaurant/quince,"An air of refinement infuses this dining room,..."


In [103]:
# testdf = pd.read_pickle('data/michelin_df.pickle')

In [99]:
# testdf = pd.read_pickle('data/michelin_df.pickle')

In [ ]:
# df['data'].str.extract(',:}')

In [ ]:
# example_string = re.sub(r'[{}]', '', example_string)

In [46]:
# df.to_pickle('data/testdf.pickle')

In [47]:
# dftest = pd.read_pickle('data/testdf.pickle')

In [51]:
# dftest[:5].to_pickle('data/dftest5.pickle')

In [52]:
# dftest = pd.read_pickle('michelin_df.pickle')